# Initialization

In [2]:
import os
import numpy as np
import pickle
import json
import pandas as pd
from grabbit import Layout
from mne import read_epochs, grand_average, write_evokeds, read_evokeds
from mne.viz import plot_compare_evokeds
from msit import CH_NAMES
import matplotlib.pyplot as plt

# load subject names to process
pipeline_root = '../data/derivatives/eeg_sensor_erps'
layout = Layout('../data', '../data/grabbit_config.json')
subjects = np.array(sorted(layout.get(target='subject', 
                                      modality='eeg',
                                      return_type='id')))

# load eeg configurations
with open('../config.json') as f:    
    config = json.load(f)

# load behavior
behavior = pd.read_csv('../data/derivatives/behavior/group_data.tsv', 
                       na_values='n/a', sep='\t') 
behavior = behavior[behavior.modality == 'eeg']

# Make directory structure
pipeline_root = '../data/derivatives/eeg_sensor_erps'
if not os.path.exists(pipeline_root):
    os.makedirs(pipeline_root)

layout = Layout('../data', '../data/grabbit_config.json')
subjects = layout.get(target='subject', modality='eeg', return_type='id')

for subject in subjects + ['group']:
    if not os.path.exists('%s/%s/erps' % (pipeline_root, subject)):
        os.makedirs('%s/%s/erps' % (pipeline_root, subject))

# Make Evoked Data

In [3]:
for epo_type, epo_times in zip(config['epoch_types'], 
                               config['epoch_times']):
    
    evokeds = {'incongruent': [], 'congruent': []}
    
    for subject in subjects:
        if subject == 'sub-hc013':
            continue
        print(subject)

        sub_behavior = behavior[behavior.participant_id == subject]
        sub_behavior = sub_behavior[sub_behavior.no_response != 1].reset_index()
    
        # load, crop, and interpolate cleaned epochs
        epo_file = layout.get(subject=subject, 
                              derivative='eeg_preprocessing', 
                              extensions='%s_cleaned-epo.fif' % epo_type)[0].filename
        epochs = read_epochs(epo_file, verbose=False)
        epochs.crop(epo_times[0], epo_times[1])
        epochs.interpolate_bads(reset_bads=True)

        # drop bad eeg trials from behavior
        ar_file = layout.get(subject=subject, 
                             derivative='eeg_preprocessing', 
                             extensions='%s_ar.pkl' % epo_type)[0].filename
        ar = pickle.load(open(ar_file, 'r'))
        sub_behavior = sub_behavior.drop(ar.bad_epochs_idx).reset_index()
        
        # drop bad behavior trials from eeg and behavior
        behavior_exclude = np.where(np.sum(sub_behavior[['fast_rt', 'error', 
                                                         'post_error']], 
                                           axis=1))[0]
        sub_behavior = sub_behavior.drop(behavior_exclude)
        epochs.drop(behavior_exclude)
        
        # inject event labels
        events = sub_behavior.trial_type.astype('category').cat.codes
        if subject == 'sub-pp012':
            events = events[1:]
        epochs.event_id = {'incongruent': 1, 'congruent': 0}
        epochs.events[:, -1] = events
        
        # write evokeds to file
        evos = [epochs['incongruent'].average(), epochs['congruent'].average()]
        write_evokeds('%s/%s/%s_%s-ave.fif' % (pipeline_root,
                                               subject, subject,
                                               epo_type), evos)
        # accumulate evokeds for group average
        evokeds['incongruent'].append(evos[0])
        evokeds['congruent'].append(evos[1])
    
    # calculate group average and save
    evos = [grand_average(evokeds['incongruent']),
            grand_average(evokeds['congruent'])]
    write_evokeds('%s/group/group_%s-ave.fif' % (pipeline_root,
                                                 epo_type), evos)


print('Done!')

sub-hc001
Computing interpolation matrix from 60 sensor positions
Interpolating 10 sensors


/autofs/space/cassia_001/users/matt/software/anaconda2.7/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


sub-hc002
Computing interpolation matrix from 61 sensor positions
Interpolating 9 sensors
sub-hc003
Computing interpolation matrix from 67 sensor positions
Interpolating 3 sensors
sub-hc004
Computing interpolation matrix from 68 sensor positions
Interpolating 2 sensors
sub-hc005
Computing interpolation matrix from 65 sensor positions
Interpolating 5 sensors
sub-hc006
Computing interpolation matrix from 65 sensor positions
Interpolating 5 sensors
sub-hc007
Computing interpolation matrix from 62 sensor positions
Interpolating 8 sensors
sub-hc008
Computing interpolation matrix from 64 sensor positions
Interpolating 6 sensors
sub-hc009
Computing interpolation matrix from 63 sensor positions
Interpolating 7 sensors
sub-hc010
Computing interpolation matrix from 64 sensor positions
Interpolating 6 sensors
sub-hc011
Computing interpolation matrix from 66 sensor positions
Interpolating 4 sensors
sub-hc012
Computing interpolation matrix from 64 sensor positions
Interpolating 6 sensors
sub-hc014


Interpolating 7 sensors
sub-hc042
Computing interpolation matrix from 62 sensor positions
Interpolating 8 sensors
sub-hc044
Computing interpolation matrix from 62 sensor positions
Interpolating 8 sensors
sub-hc045
Computing interpolation matrix from 66 sensor positions
Interpolating 4 sensors
sub-pp001
Computing interpolation matrix from 61 sensor positions
Interpolating 9 sensors
sub-pp002
Computing interpolation matrix from 64 sensor positions
Interpolating 6 sensors
sub-pp003
Computing interpolation matrix from 68 sensor positions
Interpolating 2 sensors
sub-pp004
Computing interpolation matrix from 65 sensor positions
Interpolating 5 sensors
sub-pp005
Computing interpolation matrix from 56 sensor positions
Interpolating 14 sensors
sub-pp006
Computing interpolation matrix from 63 sensor positions
Interpolating 7 sensors
sub-pp007
Computing interpolation matrix from 65 sensor positions
Interpolating 5 sensors
sub-pp008
Computing interpolation matrix from 66 sensor positions
Interpola

# Plot Congruent vs. Incongruent ERPs

In [6]:
for subject in list(subjects) + ['group']:
    print(subject)

    for ch in CH_NAMES: 

        fig, axs = plt.subplots(1, 2, figsize=(24, 8))

        for i, epo_type in enumerate(config['epoch_types']):

            ax = axs[i]

            evokeds = read_evokeds('%s/%s/%s_%s-ave.fif' % (pipeline_root,
                                                            subject,
                                                            subject,
                                                            epo_type),
                                  verbose=False)
            pick = evokeds[0].ch_names.index(ch)
            evokeds = dict((config['conditions'][i], evokeds[i]) 
                           for i in range(len(evokeds)))
            plot_compare_evokeds(evokeds, picks=pick, axes=ax, show=False)
            ax.set_title('%s-locked' % epo_type)

        plt.suptitle('%s %s ERP' % (subject, ch))
        fig.savefig('%s/%s/erps/%s_%s_erp.png' % (pipeline_root, subject,
                                                  subject, ch))
        plt.close(fig)
        
print('Done!')

sub-hc001
sub-hc002
sub-hc003
sub-hc004


KeyboardInterrupt: 

In [9]:
from ipywidgets import interact

def plot_erp(subject, ch):

    fig, axs = plt.subplots(1, 2, figsize=(24, 8))

    for i, epo_type in enumerate(config['epoch_types']):

        ax = axs[i]

        evokeds = read_evokeds('%s/%s/%s_%s-ave.fif' % (pipeline_root,
                                                        subject,
                                                        subject,
                                                        epo_type),
                              verbose=False)
        pick = evokeds[0].ch_names.index(ch)
        evokeds = dict((config['conditions'][i], evokeds[i]) 
                       for i in range(len(evokeds)))
        plot_compare_evokeds(evokeds, picks=pick, axes=ax, show=False)
        ax.set_title('%s-locked' % epo_type)

    plt.suptitle('%s %s ERP' % (subject, ch))
    plt.show()

interact(plot_erp, subject=subjects + ['group'], ch=CH_NAMES)

Widget Javascript not detected.  It may not be installed or enabled properly.


<function __main__.plot_erp>